# Case Study: 코호트 리텐션 분석 — E-commerce 고객 생존 분석

> **데이터**: [UCI Online Retail Dataset](https://archive.ics.uci.edu/dataset/352/online+retail) (CC BY 4.0, 541K transactions, 4,372 customers)
>
> **요약**: UK 기반 온라인 리테일 회사의 1년간 거래 데이터(2010.12~2011.12)를 분석합니다.
> 단순 매출 추이를 넘어, **월별 코호트 리텐션 → 고객 생존 곡선 → 코호트별 LTV 추정**을 통해
> "어떤 코호트의 고객이 가장 가치 있는가?"에 답합니다.

---

## 이 분석의 구조

1. **데이터 탐색** — 541K 거래의 패턴 파악 + 데이터 정제
2. **코호트 생성** — 고객별 첫 구매 월 기준 코호트 배정
3. **리텐션 분석** — 월별 재구매 리텐션 히트맵 + 곡선
4. **매출 분석** — 코호트별 ARPU / 누적 LTV 추이
5. **세그먼트 비교** — 국가별 / 주문 금액 세그먼트별 리텐션 차이
6. **비즈니스 권장** — CRM 개입 시점 + 리텐션 개선 전략

In [ ]:
import sys, os, warnings
sys.path.insert(0, os.path.abspath('..'))
warnings.filterwarnings('ignore', category=FutureWarning)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sns

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({'figure.figsize': (12, 5), 'font.size': 11})

# --- Data Loading ---
# UCI Online Retail Dataset
# Download: https://archive.ics.uci.edu/dataset/352/online+retail
DATA_FILE = 'online_retail.csv'

if os.path.exists(DATA_FILE):
    df = pd.read_csv(DATA_FILE, encoding='ISO-8859-1', parse_dates=['InvoiceDate'])
elif os.path.exists('Online Retail.xlsx'):
    df = pd.read_excel('Online Retail.xlsx', parse_dates=['InvoiceDate'])
    df.to_csv(DATA_FILE, index=False)  # cache as CSV
else:
    # Fallback: download from UCI
    url = 'https://archive.ics.uci.edu/static/public/352/online+retail.zip'
    import zipfile, io, urllib.request
    print(f'Downloading from {url}...')
    resp = urllib.request.urlopen(url)
    with zipfile.ZipFile(io.BytesIO(resp.read())) as z:
        xlsx_name = [n for n in z.namelist() if n.endswith('.xlsx')][0]
        with z.open(xlsx_name) as f:
            df = pd.read_excel(f, parse_dates=['InvoiceDate'])
    df.to_csv(DATA_FILE, index=False)
    print('Downloaded and cached as online_retail.csv')

print(f'Dataset: {len(df):,} rows, {df.shape[1]} columns')
print(f'Columns: {list(df.columns)}')
df.head()

---

## 1. 탐색적 데이터 분석 (EDA)

In [ ]:
# === 데이터 정제 ===
print('=== Raw Data Summary ===')
print(f'  Total rows:       {len(df):>10,}')
print(f'  Unique invoices:  {df["InvoiceNo"].nunique():>10,}')
print(f'  Unique customers: {df["CustomerID"].nunique():>10,}')
print(f'  Unique products:  {df["StockCode"].nunique():>10,}')
print(f'  Date range:       {df["InvoiceDate"].min().date()} to {df["InvoiceDate"].max().date()}')
print(f'\n=== Missing Values ===')
missing = df.isnull().sum()
for col in missing[missing > 0].index:
    print(f'  {col:>15s}: {missing[col]:>6,} ({missing[col]/len(df):.1%})')

# 정제: CustomerID 결측 제거, 취소 주문(C로 시작) 제거, 수량/금액 양수만
df_clean = df.dropna(subset=['CustomerID']).copy()
df_clean['CustomerID'] = df_clean['CustomerID'].astype(int)
df_clean = df_clean[~df_clean['InvoiceNo'].astype(str).str.startswith('C')]  # 취소 제거
df_clean = df_clean[(df_clean['Quantity'] > 0) & (df_clean['UnitPrice'] > 0)]
df_clean['Revenue'] = df_clean['Quantity'] * df_clean['UnitPrice']

print(f'\n=== After Cleaning ===')
print(f'  Rows:     {len(df_clean):>10,} ({len(df_clean)/len(df):.1%} retained)')
print(f'  Customers: {df_clean["CustomerID"].nunique():>10,}')
print(f'  Revenue:   ${df_clean["Revenue"].sum():>12,.0f}')

In [ ]:
# === 4-panel EDA ===
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# (1) Monthly revenue trend
monthly_rev = df_clean.set_index('InvoiceDate').resample('M')['Revenue'].sum()
axes[0, 0].plot(monthly_rev.index, monthly_rev.values, 'o-', color='#6366f1', linewidth=2)
axes[0, 0].fill_between(monthly_rev.index, monthly_rev.values, alpha=0.1, color='#6366f1')
axes[0, 0].set_title('(1) Monthly Revenue Trend')
axes[0, 0].set_ylabel('Revenue (GBP)')
axes[0, 0].yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'\u00a3{x:,.0f}'))

# (2) Customer distribution by country (top 10)
country_counts = df_clean.groupby('Country')['CustomerID'].nunique().sort_values(ascending=False).head(10)
colors_bar = ['#6366f1' if c == 'United Kingdom' else '#94a3b8' for c in country_counts.index]
axes[0, 1].barh(country_counts.index[::-1], country_counts.values[::-1], color=colors_bar[::-1], edgecolor='white')
axes[0, 1].set_title('(2) Customers by Country (Top 10)')
axes[0, 1].set_xlabel('Unique Customers')

# (3) Order value distribution
order_values = df_clean.groupby('InvoiceNo')['Revenue'].sum()
axes[1, 0].hist(order_values[order_values < 500], bins=80, color='#6366f1', edgecolor='none', alpha=0.7)
axes[1, 0].axvline(order_values.median(), color='#f59e0b', linestyle='--', linewidth=2,
                    label=f'Median: \u00a3{order_values.median():.0f}')
axes[1, 0].set_title('(3) Order Value Distribution (< \u00a3500)')
axes[1, 0].set_xlabel('Order Value (GBP)')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].legend()

# (4) Purchase frequency per customer
purchase_freq = df_clean.groupby('CustomerID')['InvoiceNo'].nunique()
axes[1, 1].hist(purchase_freq[purchase_freq <= 30], bins=30, color='#22c55e', edgecolor='none', alpha=0.7)
axes[1, 1].axvline(purchase_freq.median(), color='#f59e0b', linestyle='--', linewidth=2,
                    label=f'Median: {purchase_freq.median():.0f} orders')
axes[1, 1].set_title('(4) Purchase Frequency per Customer')
axes[1, 1].set_xlabel('Number of Orders')
axes[1, 1].set_ylabel('Customers')
axes[1, 1].legend()

plt.suptitle('Exploratory Data Analysis \u2014 Online Retail (541K Transactions)',
             fontsize=14, fontweight='bold', y=1.01)
plt.tight_layout()
plt.savefig('cohort_eda.png', dpi=150, bbox_inches='tight')
plt.show()

### EDA 발견

1. **매출 성장 추세**: 2011년 하반기로 갈수록 매출 증가 — 계절성(연말 효과)인지 확인 필요
2. **UK 집중**: 고객의 대부분이 UK. 국가별 리텐션 차이를 확인할 것
3. **주문 금액 분포**: 중앙값 기준 비교적 저가 주문 — 소수의 대형 주문이 매출을 끌어올릴 가능성
4. **구매 빈도**: 1회 구매 고객이 가장 많음 — 리텐션 개선의 여지가 큼

---

## 2. 코호트 생성

고객의 **첫 구매 월**을 기준으로 코호트를 배정합니다.

In [ ]:
# === 코호트 배정 ===
df_clean['InvoiceMonth'] = df_clean['InvoiceDate'].dt.to_period('M')

# 고객별 첫 구매 월
cohort_month = df_clean.groupby('CustomerID')['InvoiceMonth'].min().rename('CohortMonth')
df_clean = df_clean.merge(cohort_month, on='CustomerID')

# 코호트 인덱스 (첫 구매 후 몇 개월)
df_clean['CohortIndex'] = (df_clean['InvoiceMonth'] - df_clean['CohortMonth']).apply(lambda x: x.n)

# 코호트별 크기
cohort_sizes = cohort_month.value_counts().sort_index()
print('=== Cohort Sizes (by first purchase month) ===')
for period, count in cohort_sizes.items():
    print(f'  {period}: {count:>5,} new customers')
print(f'\n  Total cohorts: {len(cohort_sizes)}')
print(f'  Total customers: {cohort_month.nunique():,}')

---

## 3. 리텐션 분석

In [ ]:
# === 리텐션 테이블 생성 ===
# 각 (코호트, 코호트인덱스)에서 구매한 고유 고객 수
cohort_data = (
    df_clean.groupby(['CohortMonth', 'CohortIndex'])['CustomerID']
    .nunique()
    .reset_index(name='Customers')
)

# 피봇: 행=코호트, 열=월 인덱스, 값=고객 수
cohort_pivot = cohort_data.pivot(index='CohortMonth', columns='CohortIndex', values='Customers')

# 코호트 크기 (Month 0)
cohort_size = cohort_pivot.iloc[:, 0]

# 리텐션율 = 해당 월 고객 수 / Month 0 고객 수
retention = cohort_pivot.divide(cohort_size, axis=0)

print('=== Retention Table (first 6 months) ===')
print(retention.iloc[:, :7].round(3).to_string())

In [ ]:
# === 핵심 시각화 #1: 리텐션 히트맵 ===
fig, ax = plt.subplots(figsize=(14, 8))

# Month 0 (100%) 제외 — 리텐션 변화에 집중
retention_display = retention.iloc[:, 1:]  # Month 1부터

# 코호트 라벨: 기간 + 크기
y_labels = [f'{str(p)} (n={cohort_size[p]:,})' for p in retention_display.index]

sns.heatmap(
    retention_display.values * 100,
    annot=True, fmt='.0f', cmap='YlOrRd_r',
    xticklabels=[f'M+{i}' for i in range(1, retention_display.shape[1] + 1)],
    yticklabels=y_labels,
    vmin=0, vmax=50,
    linewidths=0.5, linecolor='white',
    cbar_kws={'label': 'Retention Rate (%)', 'format': '%.0f%%'},
    ax=ax
)

ax.set_title('Cohort Retention Heatmap \u2014 Monthly Repurchase Rate\n'
             '(% of cohort who made a purchase in each subsequent month)',
             fontsize=13, fontweight='bold', pad=15)
ax.set_xlabel('Months After First Purchase')
ax.set_ylabel('Cohort (First Purchase Month)')

plt.tight_layout()
plt.savefig('cohort_retention_heatmap.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# === 핵심 시각화 #2: 리텐션 곡선 ===
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Left: 전체 코호트 리텐션 곡선 (개별)
colors_cohort = plt.cm.viridis(np.linspace(0.2, 0.9, len(retention)))
for i, (cohort, row) in enumerate(retention.iterrows()):
    values = row.dropna().values * 100
    if len(values) >= 3:  # 최소 3개월 이상 데이터
        axes[0].plot(range(len(values)), values, 'o-', color=colors_cohort[i],
                     alpha=0.5, markersize=3, linewidth=1, label=str(cohort))

axes[0].set_title('Individual Cohort Retention Curves')
axes[0].set_xlabel('Months After First Purchase')
axes[0].set_ylabel('Retention Rate (%)')
axes[0].set_ylim(0, 105)
axes[0].legend(fontsize=7, ncol=2, loc='upper right')

# Right: 평균 리텐션 곡선 + 95% CI
avg_retention = retention.mean(axis=0) * 100
std_retention = retention.std(axis=0) * 100
n_cohorts = retention.notna().sum(axis=0)
ci_95 = 1.96 * std_retention / np.sqrt(n_cohorts)

x = range(len(avg_retention))
axes[1].plot(x, avg_retention, 'o-', color='#6366f1', linewidth=2.5, markersize=6, label='Average')
axes[1].fill_between(x, avg_retention - ci_95, avg_retention + ci_95,
                      alpha=0.2, color='#6366f1', label='95% CI')

# Annotate key points
for i in [0, 1, 2, 3]:
    if i < len(avg_retention):
        axes[1].annotate(f'{avg_retention.iloc[i]:.1f}%',
                         (i, avg_retention.iloc[i]),
                         textcoords='offset points', xytext=(0, 12),
                         ha='center', fontweight='bold', fontsize=10)

axes[1].set_title('Average Retention Curve (All Cohorts)')
axes[1].set_xlabel('Months After First Purchase')
axes[1].set_ylabel('Retention Rate (%)')
axes[1].set_ylim(0, 105)
axes[1].legend()

plt.suptitle('Cohort Retention Curves \u2014 When Do Customers Drop Off?',
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('cohort_retention_curves.png', dpi=150, bbox_inches='tight')
plt.show()

# 리텐션 요약 출력
print('=== Average Retention by Month ===')
for i, val in enumerate(avg_retention):
    drop = '' if i == 0 else f' (\u0394{avg_retention.iloc[i] - avg_retention.iloc[i-1]:+.1f}%p)'
    print(f'  Month {i}: {val:>5.1f}%{drop}')

### 리텐션 분석 발견

1. **Month 0 → 1 드랍이 가장 급격**: 첫 달 이후 리텐션이 크게 하락 — CRM 개입의 핵심 타이밍
2. **Month 3 이후 안정화**: 3개월 이상 유지된 고객은 장기 고객이 될 가능성 높음
3. **코호트 간 편차**: 일부 코호트는 초기 리텐션이 월등 — 시즌, 프로모션 영향 가능

---

## 4. 매출 분석 — 코호트별 LTV

In [ ]:
# === 코호트별 ARPU (Average Revenue Per User) ===
cohort_revenue = (
    df_clean.groupby(['CohortMonth', 'CohortIndex'])['Revenue']
    .sum()
    .reset_index(name='TotalRevenue')
)

revenue_pivot = cohort_revenue.pivot(index='CohortMonth', columns='CohortIndex', values='TotalRevenue')

# ARPU = 월별 매출 / 코호트 크기
arpu = revenue_pivot.divide(cohort_size, axis=0)

# 누적 LTV
cumulative_ltv = arpu.cumsum(axis=1)

print('=== Average Revenue Per User (ARPU) by Cohort Month ===')
print(arpu.iloc[:, :7].round(1).to_string())
print(f'\n=== Cumulative LTV at Month 6 (top 5 cohorts) ===')
ltv_m6 = cumulative_ltv.iloc[:, 6].dropna().sort_values(ascending=False)
for cohort, ltv in ltv_m6.head().items():
    print(f'  {cohort}: \u00a3{ltv:,.0f}')

In [ ]:
# === 핵심 시각화 #3: 코호트별 누적 LTV ===
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Left: 코호트별 누적 LTV 곡선
for i, (cohort, row) in enumerate(cumulative_ltv.iterrows()):
    values = row.dropna().values
    if len(values) >= 4:
        axes[0].plot(range(len(values)), values, 'o-', color=colors_cohort[i],
                     alpha=0.6, markersize=3, linewidth=1.5, label=str(cohort))

axes[0].set_title('Cumulative LTV by Cohort')
axes[0].set_xlabel('Months After First Purchase')
axes[0].set_ylabel('Cumulative Revenue per User (\u00a3)')
axes[0].legend(fontsize=7, ncol=2, loc='upper left')

# Right: 평균 누적 LTV + Month 0 ARPU vs Long-term LTV
avg_ltv = cumulative_ltv.mean(axis=0)
avg_arpu = arpu.mean(axis=0)

x = range(len(avg_ltv))
axes[1].bar(x, avg_arpu, color='#6366f1', alpha=0.4, label='Monthly ARPU')
axes[1].plot(x, avg_ltv, 'o-', color='#f59e0b', linewidth=2.5, markersize=6, label='Cumulative LTV')

for i in [0, 3, 6]:
    if i < len(avg_ltv):
        axes[1].annotate(f'\u00a3{avg_ltv.iloc[i]:,.0f}',
                         (i, avg_ltv.iloc[i]),
                         textcoords='offset points', xytext=(0, 12),
                         ha='center', fontweight='bold', fontsize=10)

axes[1].set_title('Average Cumulative LTV + Monthly ARPU')
axes[1].set_xlabel('Months After First Purchase')
axes[1].set_ylabel('Revenue per User (\u00a3)')
axes[1].legend()

plt.suptitle('Customer Lifetime Value (LTV) Analysis',
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('cohort_ltv.png', dpi=150, bbox_inches='tight')
plt.show()

---

## 5. 세그먼트 비교 — 지역별 & 주문 금액별

고객 리텐션이 **지역(UK vs Non-UK)**과 **주문 금액 세그먼트(High/Mid/Low)**에 따라 어떻게 달라지는지 비교합니다.

In [ ]:
# === UK vs Non-UK 리텐션 비교 ===
df_clean['IsUK'] = (df_clean['Country'] == 'United Kingdom')

retention_by_region = {}
for region, label in [(True, 'UK'), (False, 'Non-UK')]:
    subset = df_clean[df_clean['IsUK'] == region]
    cohort_data_r = (
        subset.groupby(['CohortMonth', 'CohortIndex'])['CustomerID']
        .nunique()
        .reset_index(name='Customers')
    )
    pivot_r = cohort_data_r.pivot(index='CohortMonth', columns='CohortIndex', values='Customers')
    size_r = pivot_r.iloc[:, 0]
    ret_r = pivot_r.divide(size_r, axis=0)
    retention_by_region[label] = ret_r

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

for idx, (label, ret_r) in enumerate(retention_by_region.items()):
    avg = ret_r.mean(axis=0) * 100
    std = ret_r.std(axis=0) * 100
    n = ret_r.notna().sum(axis=0)
    ci = 1.96 * std / np.sqrt(n)
    color = '#6366f1' if label == 'UK' else '#f59e0b'
    x = range(len(avg))
    
    # Both on left panel for comparison
    axes[0].plot(x, avg, 'o-', color=color, linewidth=2, markersize=5, label=label)
    axes[0].fill_between(x, avg - ci, avg + ci, alpha=0.15, color=color)

axes[0].set_title('UK vs Non-UK: Average Retention Curve')
axes[0].set_xlabel('Months After First Purchase')
axes[0].set_ylabel('Retention Rate (%)')
axes[0].set_ylim(0, 105)
axes[0].legend(fontsize=11)

# Right: Month 1~3 리텐션 비교 바 차트
months_compare = [1, 2, 3, 4, 5, 6]
uk_vals = [retention_by_region['UK'].mean(axis=0).iloc[m] * 100 if m < len(retention_by_region['UK'].mean(axis=0)) else 0 for m in months_compare]
nonuk_vals = [retention_by_region['Non-UK'].mean(axis=0).iloc[m] * 100 if m < len(retention_by_region['Non-UK'].mean(axis=0)) else 0 for m in months_compare]

x_bar = np.arange(len(months_compare))
w = 0.35
axes[1].bar(x_bar - w/2, uk_vals, w, color='#6366f1', label='UK', edgecolor='white')
axes[1].bar(x_bar + w/2, nonuk_vals, w, color='#f59e0b', label='Non-UK', edgecolor='white')
axes[1].set_xticks(x_bar)
axes[1].set_xticklabels([f'M+{m}' for m in months_compare])
axes[1].set_title('UK vs Non-UK: Monthly Retention Comparison')
axes[1].set_ylabel('Retention Rate (%)')
axes[1].legend()

plt.suptitle('Segment Comparison \u2014 Does Geography Affect Retention?',
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('cohort_segment_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

# 정량 비교
print('=== UK vs Non-UK Retention (Month 1-6) ===')
for m in months_compare:
    uk_r = retention_by_region['UK'].mean(axis=0)
    nuk_r = retention_by_region['Non-UK'].mean(axis=0)
    if m < len(uk_r) and m < len(nuk_r):
        diff = (uk_r.iloc[m] - nuk_r.iloc[m]) * 100
        print(f'  Month {m}: UK={uk_r.iloc[m]*100:.1f}%, Non-UK={nuk_r.iloc[m]*100:.1f}%, Diff={diff:+.1f}%p')

In [ ]:
# === 주문 금액 세그먼트별 리텐션 비교 ===
# 고객별 평균 주문 금액으로 세그먼트 분류
customer_avg_order = (
    df_clean.groupby('CustomerID')
    .agg(avg_order_value=('Revenue', 'mean'), total_revenue=('Revenue', 'sum'))
)

# 3분위로 세그먼트: Low / Mid / High
customer_avg_order['Segment'] = pd.qcut(
    customer_avg_order['avg_order_value'],
    q=3,
    labels=['Low-Value', 'Mid-Value', 'High-Value']
)

print('=== Customer Segments by Avg Order Value ===')
for seg in ['Low-Value', 'Mid-Value', 'High-Value']:
    subset = customer_avg_order[customer_avg_order['Segment'] == seg]
    print(f'  {seg:>12s}: {len(subset):>5,} customers, '
          f'Avg Order £{subset["avg_order_value"].mean():>7,.1f}, '
          f'Avg Total £{subset["total_revenue"].mean():>8,.0f}')

# 세그먼트를 df_clean에 조인
df_clean = df_clean.merge(
    customer_avg_order[['Segment']],
    on='CustomerID',
    how='left'
)

# 세그먼트별 리텐션 계산
segment_colors = {'Low-Value': '#22c55e', 'Mid-Value': '#6366f1', 'High-Value': '#f59e0b'}
retention_by_segment = {}

for seg in ['Low-Value', 'Mid-Value', 'High-Value']:
    subset = df_clean[df_clean['Segment'] == seg]
    cd = (
        subset.groupby(['CohortMonth', 'CohortIndex'])['CustomerID']
        .nunique()
        .reset_index(name='Customers')
    )
    piv = cd.pivot(index='CohortMonth', columns='CohortIndex', values='Customers')
    sz = piv.iloc[:, 0]
    ret = piv.divide(sz, axis=0)
    retention_by_segment[seg] = ret

# === 시각화: 세그먼트별 리텐션 곡선 비교 ===
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Left: 리텐션 곡선 비교
for seg, ret in retention_by_segment.items():
    avg = ret.mean(axis=0) * 100
    std = ret.std(axis=0) * 100
    n = ret.notna().sum(axis=0)
    ci = 1.96 * std / np.sqrt(n)
    x = range(len(avg))
    axes[0].plot(x, avg, 'o-', color=segment_colors[seg], linewidth=2, markersize=5, label=seg)
    axes[0].fill_between(x, avg - ci, avg + ci, alpha=0.12, color=segment_colors[seg])

axes[0].set_title('Retention by Order Value Segment')
axes[0].set_xlabel('Months After First Purchase')
axes[0].set_ylabel('Retention Rate (%)')
axes[0].set_ylim(0, 105)
axes[0].legend(fontsize=11)

# Right: Month 1~6 리텐션 비교 (Grouped Bar)
months_compare = [1, 2, 3, 4, 5, 6]
x_bar = np.arange(len(months_compare))
n_seg = len(retention_by_segment)
w = 0.25

for i, (seg, ret) in enumerate(retention_by_segment.items()):
    avg = ret.mean(axis=0) * 100
    vals = [avg.iloc[m] if m < len(avg) else 0 for m in months_compare]
    offset = (i - n_seg / 2 + 0.5) * w
    axes[1].bar(x_bar + offset, vals, w, color=segment_colors[seg], label=seg, edgecolor='white')

axes[1].set_xticks(x_bar)
axes[1].set_xticklabels([f'M+{m}' for m in months_compare])
axes[1].set_title('Monthly Retention by Order Value Segment')
axes[1].set_ylabel('Retention Rate (%)')
axes[1].legend()

plt.suptitle('Does Order Value Predict Customer Retention?',
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('cohort_segment_ordervalue.png', dpi=150, bbox_inches='tight')
plt.show()

# 정량 비교
print('\n=== Order Value Segment Retention (Month 1-6) ===')
for m in months_compare:
    print(f'  Month {m}:', end='')
    for seg, ret in retention_by_segment.items():
        avg = ret.mean(axis=0) * 100
        if m < len(avg):
            print(f'  {seg}={avg.iloc[m]:.1f}%', end='')
    print()

---

## 6. 비즈니스 인사이트 & 권장 사항

In [ ]:
# === 정량 요약 ===
avg_ret = retention.mean(axis=0) * 100
avg_ltv_vals = cumulative_ltv.mean(axis=0)

print('=' * 65)
print('  COHORT RETENTION ANALYSIS \u2014 EXECUTIVE SUMMARY')
print('=' * 65)

print(f'\n[1] Key Retention Metrics')
if len(avg_ret) > 1:
    print(f'    Month 0 \u2192 1 retention:  {avg_ret.iloc[1]:.1f}%')
    print(f'    Month 0 \u2192 1 drop:       {100 - avg_ret.iloc[1]:.1f}%p lost')
if len(avg_ret) > 3:
    print(f'    Month 3 retention:       {avg_ret.iloc[3]:.1f}%')
if len(avg_ret) > 6:
    print(f'    Month 6 retention:       {avg_ret.iloc[6]:.1f}%')

print(f'\n[2] Revenue Impact')
total_customers = df_clean['CustomerID'].nunique()
total_revenue = df_clean['Revenue'].sum()
print(f'    Total customers: {total_customers:,}')
print(f'    Total revenue:   \u00a3{total_revenue:,.0f}')
print(f'    Average LTV (Month 0): \u00a3{avg_ltv_vals.iloc[0]:,.0f}')
if len(avg_ltv_vals) > 6:
    print(f'    Average LTV (Month 6): \u00a3{avg_ltv_vals.iloc[6]:,.0f}')
    ltv_growth = (avg_ltv_vals.iloc[6] / avg_ltv_vals.iloc[0] - 1) * 100
    print(f'    LTV growth (0\u21926): +{ltv_growth:.0f}%')

print(f'\n[3] Churn Risk Segments')
one_time = purchase_freq[purchase_freq == 1].count()
print(f'    One-time buyers:    {one_time:,} ({one_time/total_customers:.1%})')
print(f'    Repeat buyers:      {total_customers - one_time:,} ({(total_customers - one_time)/total_customers:.1%})')

print(f'\n[4] Revenue at Risk')
if len(avg_ret) > 1:
    lost_pct = (100 - avg_ret.iloc[1]) / 100
    lost_customers = int(total_customers * lost_pct)
    potential_rev = lost_customers * avg_ltv_vals.iloc[0]
    print(f'    Month 1 churned customers: ~{lost_customers:,}')
    print(f'    If 10% recovered (via CRM): ~{int(lost_customers * 0.1):,} customers')
    print(f'    Estimated recovered revenue: \u00a3{lost_customers * 0.1 * avg_ltv_vals.iloc[0]:,.0f}')

### PM에게 전달하는 분석 결과

> **1. 첫 구매 후 1개월이 가장 위험한 구간입니다.**
>
> 코호트 평균 Month 0→1 리텐션이 가장 크게 하락합니다.
> 이 구간에 CRM 개입(재구매 쿠폰, 교차 판매 이메일)을 집중하면
> 리텐션 개선 ROI가 가장 높습니다.
>
> **2. 3개월 이상 유지된 고객은 장기 고객이 될 가능성이 높습니다.**
>
> Month 3 이후 리텐션 곡선이 안정화됩니다.
> 이 고객군에게는 로열티 프로그램이나 VIP 혜택을 제공하여 이탈을 방지해야 합니다.
>
> **3. 권장 액션**
> - 1개월 내 재구매 유도 캠페인 A/B 테스트 (할인 vs 교차판매 vs 무료배송)
> - 3개월 생존 고객 대상 로열티 프로그램 설계
> - UK vs Non-UK 리텐션 차이 원인 조사 (배송 속도? 상품 구성?)

---

### 방법론적 한계

| 한계 | 설명 | 완화 방안 |
|------|------|----------|
| **구매 기반 리텐션** | 방문/로그인 기반이 아닌 구매 기반이므로 실제 활동 리텐션보다 낮게 측정됨 | 로그인 데이터가 있으면 DAU/WAU 기반 리텐션 병행 |
| **세션 정의** | 월별 측정이므로 월말 구매 고객이 불리할 수 있음 | 주별 코호트로 보완 |
| **생존 편향** | 오래 남은 코호트일수록 관찰 기간이 김 | 동일 기간 비교 시 주의 |

### Reference

- Dataset: Chen, D. (2012). *Online Retail*. UCI Machine Learning Repository. CC BY 4.0.
- Fader, P. & Hardie, B. (2005). "Counting Your Customers the Easy Way." *Marketing Science*, 24(2).
- Reichheld, F. (2003). "The One Number You Need to Grow." *Harvard Business Review*.